<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-简化版PageRank模型" data-toc-modified-id="1-简化版PageRank模型-1">1 简化版PageRank模型</a></span><ul class="toc-item"><li><span><a href="#1.1--构建转移矩阵" data-toc-modified-id="1.1--构建转移矩阵-1.1">1.1  构建转移矩阵</a></span></li><li><span><a href="#1.2-构建转换过程" data-toc-modified-id="1.2-构建转换过程-1.2">1.2 构建转换过程</a></span></li><li><span><a href="#1.3-尝试第一次转换" data-toc-modified-id="1.3-尝试第一次转换-1.3">1.3 尝试第一次转换</a></span></li><li><span><a href="#1.4-循环迭代，找到最终影响力矩阵" data-toc-modified-id="1.4-循环迭代，找到最终影响力矩阵-1.4">1.4 循环迭代，找到最终影响力矩阵</a></span></li></ul></li><li><span><a href="#2--随机浏览模型" data-toc-modified-id="2--随机浏览模型-2">2  随机浏览模型</a></span><ul class="toc-item"><li><span><a href="#2.1-引入阻尼因子-用随机浏览模型迭代" data-toc-modified-id="2.1-引入阻尼因子-用随机浏览模型迭代-2.1">2.1 引入阻尼因子 用随机浏览模型迭代</a></span></li></ul></li><li><span><a href="#3-总结：" data-toc-modified-id="3-总结：-3">3 总结：</a></span></li></ul></div>

使用Python模拟下面的PageRank计算过程，求每个节点的影响力（迭代100次）

In [10]:
import numpy as np

# 1 简化版PageRank模型

## 1.1  构建转移矩阵

ABCD有向图：

---
<img src="./pictures/ABCD有向图.png" width="40%">


---
一个网页u的影响力=所有入链集合的页面的加权影响力之和: $PR(u)=\sum_{v\epsilon B_{u}}\frac{PR(v)}{L(v)}$    

- 公式解读：
    - u：待评估的页面，
    - $B_{u}$为u的入链集合
    - 对于$B_{u}$中的任意页面v，它能给u带来的影响力是其自身的影响力PR(v)除以v页面的出链数量。即：页面v把影响力PR(v)平均分配给了它的出链。这样统计所有能给u带来链接的页面v，得到的总和即为u的影响力，即为PR(u)


---
- 转移矩阵：统计网页对于其他网页的跳转概率
    - 原因：出链会给被链接的页面赋予影响力，关键在于统计他们出链的数量
    - 行表示该节点会被其他节点跳转到它的可能性
    - 列表示该节点跳转到其他节点的概率，所以每1列的和=1。

In [21]:
# 根据上图，假设：从同一个节点跳转到其他连接的节点的概率相同，创建ABCD的转移矩阵

# A点依次跳转到ABCD的可能性: A有3个出度，分别指向BCD，跳转到3个节点的概率相同
A = np.array([0., 1 / 3, 1 / 3, 1 / 3])

# B点依次对ABCD的可能性: B有2个出度，分别指向AD，跳转到2个节点概率相同，未指向的节点跳转概率为0
B = np.array([1 / 2, 0., 0., 1 / 2])

# C点依次对ABCD的可能性: C有1个出度，指向A，跳转到A节点的概率为1，未指向的节点跳转概率为0
C = np.array([1., 0., 0., 0.])

# D点依次对ABCD的可能性: D有2个出度，分别指向BC，跳转到2个节点概率相同，未指向的节点跳转概率为0
D = np.array([0., 1 / 2, 1 / 2, 0.])

print("验证各节点跳转概率的数组是否和都为1：", sum(A), sum(B), sum(C), sum(D))


验证各节点影响力数组是否和都为1： 1.0 1.0 1.0 1.0


In [22]:
# 合并为完整转移矩阵，默认是垂直方向表示，因此最后使用T转置
transition_arr = np.array([A, B, C, D]).T
transition_arr

array([[0.        , 0.5       , 1.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

In [26]:
# 假设：4个点各自的初始影响力相同，均为1/4
w0 = np.ones((4, 1)) / 4

In [27]:
w0

array([[0.25],
       [0.25],
       [0.25],
       [0.25]])

## 1.2 构建转换过程

- 转换的目标：找到最终的影响力矩阵；
- 衡量标准 ：当转移矩阵与影响力矩阵的点乘后的影响力不在变化。

## 1.3 尝试第一次转换

In [28]:
w1 = np.dot(transition_arr, w0) 
w1

array([[0.375     ],
       [0.20833333],
       [0.20833333],
       [0.20833333]])

## 1.4 循环迭代，找到最终影响力矩阵

In [34]:
w = w0
for i in range(100):
    wi = np.dot(transition_arr, w)
    if (wi - w).sum() == 0.:
        print(f"在第{i}轮循环找到最终的w:\n{w}")
        break
    else:
        w = wi

在第52轮循环找到最终的w:
[[0.33333333]
 [0.22222222]
 [0.22222222]
 [0.22222222]]


In [32]:
np.dot(transition_arr, w)

array([[0.33333333],
       [0.22222222],
       [0.22222222],
       [0.22222222]])

In [33]:
w.sum()

1.0

可知，A的影响力最大，占0.33333333。


然而不是所有的节点都有出度和入度，由于简化版模型认为所有的节点都有出度和入度，因此在实际应用中，简化版模型不一定有作用。

- Rank Leak（等级泄露）
一个网页没有出链，就像是一个黑洞一样，吸收了别人的影响力而不释放，最终会导致其他网页的PR值为0。
- Rank Sink（等级沉没）
一个网页只有出链，没有入链。迭代下来，会导致这个网页的PR值为0


# 2  随机浏览模型
---
为了解决Rank Leak（等级泄露）和Rank Sink（等级沉没）的问题，Larry Page提出了PageRank的随机浏览模型    
假设场景：用户并不都是按照跳转链接的方式来上网，还有一种可能是不论当前处于哪个页面，都有概率访问到其他任意的页面     
引入阻尼因子：d     
通常取值为0.85（默认）    
$PR(u) = \frac{1-d}{N} + d\sum_{v\epsilon B_{u}} \frac{PR(v)}{L(v)}$

## 2.1 引入阻尼因子 用随机浏览模型迭代

In [35]:
w = w0
d = 0.85
for i in range(100):
    wi = (1 - d) / len(transition_arr) + d * np.dot(transition_arr, w)
    if (wi - w).sum() == 0.:
        print(f"第{i}轮迭代，找到最终的影响力矩阵:\n{w}")
        break
    else:
        w = wi

第3轮迭代，找到最终的影响力矩阵:
[[0.33028516]
 [0.22323828]
 [0.22323828]
 [0.22323828]]


虽然和简化模型的结果略微不同，但影响力的排序没有变化，仍然影响力最高的是A。

# 3 总结：

- PageRank模型中，先根据出链和进链得到它们的转移矩阵，令初始影响力相等，转移矩阵与影响力矩阵的点乘结果是新的影响力矩阵，循环迭代计算点乘，直到得到的影响力矩阵不在变化，我们就得到最终的影响力矩阵。
- 1中的简化版PageRank模型，没有考虑只有出度没有进度和只有进度没有出度的极端情况，所以并不适用于实际情况；
- 2中的随机浏览模型，引入阻尼因子解决了1中没考虑的问题，使得到的结果更准确。